In [1]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
)
from tqdm.auto import tqdm

In [2]:
import torch

In [3]:
import pytorch_lightning as pl

C:\Users\hotai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchmetrics\functional\text\chrf.py:43: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:68.)
  _EPS_SMOOTHING = tensor(1e-16)


In [4]:
import pandas as pd
import numpy as np
import torch
import transformers
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

In [5]:
MODEL_NAME = "minhtoan/t5-small-vietnamese-news"

# MODEL_NAME = "minhtoan/t5-small-vietnamese-news"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

In [6]:
src = 'VKS cáo buộc ông Nguyễn Thế Hiệp có sai phạm trong vụ cháy gần Bệnh viện Nhi trung ương khiến 2 người chết, thiệt hại 1,9 tỷ đồng song bị cáo khẳng định vô tội. Mức án đề nghị 9-10 năm tù với bị cáo 73 tuổi được đại diện VKSND quận Ba Đình đưa ra chiều 28/11, quy buộc phạm tội Vi phạm quy định về phòng cháy chữa cháy, theo Điều 313 Bộ luật Hình sự. VKS nhận định ông Hiệp có lỗi trong việc vận hành nhà trọ không phép, không đủ điều kiện an toàn phòng cháy chữa cháy, gây thiệt hại về tài sản và khiến hai người chết. Tuy nhiên, bị cáo chưa bồi thường. Bản luận tội nêu, tại phiên tòa hôm nay ông Hiệp "chưa tỏ thái độ ăn năn hối hận, có nhân thân đặc biệt xấu". Từ hàng chục năm trước, ông từng 11 lần bị lập danh chỉ bản về hành vi trộm cắp, năm 1985 lại nhận 18 năm tù về các tội cướp tài sản, hiếp dâm, đưa hối lộ...'# 

In [7]:
tokenized_text = tokenizer(src, return_tensors="pt")
tokenized_text

{'input_ids': tensor([[  434, 12016,  4625,   268,   758,  1514,   297,   259,  1735,   259,
         15355,  1516,  1302,   447, 41352,   885,  9474,   690,  1262,   282,
           259,  1369,   300,  1308, 24151,   276,   259,   318,  1313,   364,
          6576,   334,   300,  1375,  4693,   266,   259, 12046,   259,  2292,
          2076,  1405,   356,  1672,   562,  1878,   261,   394,  5925,   382,
          1155, 76209,   259,   270,  8679,  5013, 15482,   330,   933,  4625,
           268,  1312, 10261,   370,   355,  3902,   300,  1858,   259,   270,
          2291,   260,   352,  1690,   259,  1301,   355,  1492,  3442,   933,
           259, 97657,   259,  3488,   259,   270,  1541,   300,   908,   330,
           933,  4625,   268, 11653,   719,  1544,   266,   259,  1318,   355,
          1155,   331,  1375,   434, 12016,  9965,   819,  2011,  2215,   977,
          1253, 18218,   262,  1101,   317,  7989,  1190, 31347,   261,  5047,
           758,  1514,   297,   690,  

In [8]:
class NewsSummaryDataset(Dataset):
 def __init__(
      self,
      data: pd.DataFrame,
      tokenizer: T5Tokenizer,
      text_max_token_len: int = 512,
      summary_max_token_len: int = 128
 ):
   self.tokenizer = tokenizer
   self.data = data
   self.text_max_token_len = text_max_token_len
   self.summary_max_token_len = summary_max_token_len
 def __len__(self):
   return len(self.data)
 def __getitem__(self, index : int):
   data_row = self.data.iloc[index]

   text = data_row["Content"]
   text_encoding = tokenizer(
       text,
       max_length = self.text_max_token_len,
       padding = "max_length",
       truncation = True,
       return_attention_mask = True,
       add_special_tokens = True,
       return_tensors = "pt"
   )
   summary_encoding = tokenizer(
       data_row["Summary"],
       max_length = self.summary_max_token_len,
       padding = "max_length",
       truncation = True,
       return_attention_mask = True,
       add_special_tokens = True,
       return_tensors = "pt"
   )
   labels = summary_encoding["input_ids"]
   labels[labels == 0] = -100

   return dict(
       text = text,
       summary = data_row["Summary"],
       text_input_ids = text_encoding["input_ids"].flatten(),
       text_attention_mask = text_encoding["attention_mask"].flatten(),
       labels = labels.flatten(),
       labels_attention_mask = summary_encoding["attention_mask"].flatten()
   )

In [9]:
class NewsSummaryDataModule(pl.LightningDataModule):
   def __init__(
       self,
       train_df: pd.DataFrame,
       test_df: pd.DataFrame,
       tokenizer: T5Tokenizer,
       batch_size: int = 8 ,
       text_max_token_len: int =512,
       summary_max_token_len: int = 128
   ):
       super().__init__()
       self.train_df = train_df
       self.test_df = test_df

       self.batch_size = batch_size
       self.tokenizer = tokenizer

       self.text_max_token_len = text_max_token_len
       self.summary_max_token_len = summary_max_token_len
   def setup(self, stage = None):
      self.train_dataset = NewsSummaryDataset(
          self.train_df,
          self.tokenizer,
          self.text_max_token_len,
          self.summary_max_token_len
      )
      self.test_dataset = NewsSummaryDataset(
          self.test_df,
          self.tokenizer,
          self.text_max_token_len,
          self.summary_max_token_len
      )
   def train_dataloader(self):
      return DataLoader(
          self.train_dataset,
          batch_size = self.batch_size,
          shuffle = True,
          num_workers = 2
      )
   def val_dataloader(self):
     return DataLoader(
       self.test_dataset,
       batch_size = self.batch_size,
       shuffle = True,
       num_workers = 2
     )

In [10]:
class NewsSummaryModel(pl.LightningModule):
   def __init__(self):
     super().__init__()
     self.model =  T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict = True)
   def forward(self, input_ids, attention_mask, decoder_attention_mask, labels = None):
     output = self.model(
         input_ids,
         attention_mask = attention_mask,
         labels = labels,
         decoder_attention_mask = decoder_attention_mask
     )

     return output.loss, output.logits

   def training_step(self, batch, batch_idx):
     input_ids = batch["text_input_ids"]
     attention_mask = batch["text_attention_mask"]
     labels = batch["labels"]
     labels_attention_mask = batch["labels_attention_mask"]

     loss, outputs = self(
         input_ids = input_ids,
         attention_mask = attention_mask,
         decoder_attention_mask = labels_attention_mask,
         labels = labels
     )
     self.log("train_loss", loss, prog_bar = True, logger = True)
     return loss
   def validation_step(self, batch, batch_idx):
     input_ids = batch["text_input_ids"]
     attention_mask = batch["text_attention_mask"]
     labels = batch["labels"]
     labels_attention_mask = batch["labels_attention_mask"]

     loss, outputs = self(
         input_ids = input_ids,
         attention_mask = attention_mask,
         decoder_attention_mask = labels_attention_mask,
         labels = labels
     )
     self.log("val_loss", loss, prog_bar = True, logger = True)
     return loss

   def test_step(self, batch, batch_idx):
     input_ids = batch["text_input_ids"]
     attention_mask = batch["text_attention_mask"]
     labels = batch["labels"]
     labels_attention_mask = batch["labels_attention_mask"]

     loss, outputs = self(
         input_ids = input_ids,
         attention_mask = attention_mask,
         decoder_attention_mask = labels_attention_mask,
         labels = labels
     )

     self.log("test_loss", loss, prog_bar = True, logger = True)
     return loss

   def configure_optimizers(self):
     return AdamW(self.parameters(), lr = 0.0001)

In [11]:
trained_model = NewsSummaryModel.load_from_checkpoint(
    './best-checkpoint_cpu.ckpt'
)

C:\Users\hotai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pytorch_lightning\utilities\migration\utils.py:51: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.4, which is newer than your current Lightning version: v2.0.3
  rank_zero_warn(


In [20]:
text='''Vào giữa năm 2022, các đối thủ trong khu vực của Grab như Sea Group (công ty mẹ Shopee) và GoTo Group (công ty mẹ Gojek) đều đã trải qua nhiều đợt cắt giảm hàng ngàn nhân sự. Tuy nhiên, Tập đoàn Công nghệ Grab vẫn duy trì chính sách hạn chế sa thải và chỉ giảm tốc tuyển dụng, cũng như hợp lý hoá một số chức năng.

Vào tháng 9/2022, Alex Hungate - Giám đốc vận hành Grab tuyên bố, công ty không nằm trong xu hướng này. Vị lãnh đạo này cũng nhấn mạnh, công ty luôn thận trọng trong hoạt động tuyển dụng và chưa thấy bất cứ khó khăn nào để ngừng tuyển dụng hoặc giảm số lượng nhân viên.

Quan điểm không sa thải nhân viên được Tổng giám đốc kiêm nhà đồng sáng lập Grab Anthony Tan nhắc lại trong một bản ghi nhớ gửi nhân viên vào tháng 12/2022. Tuy nhiên, trong bản ghi nhớ này, Grab nói rằng đang tạm dừng tuyển dụng những vị trí không quan trọng và tạm dừng tăng lương cho các vị trí quản lý cấp cao.
Vào tháng 6 năm nay, Grab ra thông báo cắt giảm hơn 1.000 nhân sự. Đây là một phần trong nỗ lực quản lý kinh phí và duy trì sức cạnh tranh của công ty. Cụ thể, trong thư gửi đến nhân viên được đăng tải trên trang web, Giám đốc điều hành Grab Anthony Tan khẳng định, việc tái cấu trúc là bước đi khó khăn nhưng cần thiết. Ông Tan cho rằng, các công nghệ như trí tuệ nhân tạo (AI) tạo sinh đang phát triển với tốc độ chóng mặt. Chi phí vốn đã tăng lên, ảnh hưởng trực tiếp tới bối cảnh cạnh tranh.

Giám đốc điều hành của Grab cũng nhấn mạnh, mục tiêu chính của quyết định này là tổ chức lại công ty một cách chiến lược, để có thể đi nhanh hơn, làm việc thông minh hơn, và tái cân bằng các nguồn lực trong danh mục đầu tư để phù hợp với những chiến lược lâu dài. Grab đang trên đà hòa vốn trong năm nay dù không cần sa thải nhân viên. Năm 2020, Grab đã cắt giảm 360 nhân viên, tức khoảng 5% lực lượng lao động toàn thời gian, do ảnh hưởng của đại dịch COVID-19.

Ông Anthony Tan cũng khẳng định, công ty gọi xe và giao đồ ăn vẫn đang “đi đúng hướng” và bác bỏ thông tin kế hoạch sa thải nhân viên là để nhanh chóng đạt được lợi nhuận. Vị lãnh đạo này cho rằng, dù điều chỉnh lực lượng lao động hay không, công ty vẫn trên con đường đạt mức hòa vốn hệ số EBITDA đã điều chỉnh trong năm nay.

Lợi nhuận EBITDA đã điều chỉnh của Grab đã cải thiện 5 quý liên tiếp. Tuy nhiên, tổng giá trị hàng hóa (GMV) trong quý đầu tiên của năm nay bắt đầu có dấu hiệu đình trệ, buộc công ty phải tìm cách tăng GMV nếu muốn đạt mục tiêu hòa vốn.

“Để tận dụng những cơ hội này hiệu quả nhất, chúng tôi phải kết hợp yếu tố quy mô với khả năng triển khai nhanh chóng, tiết kiệm chi phí để có thể cung cấp dịch vụ có giá cả phải chăng hơn và thâm nhập sâu hơn vào đại chúng một cách bền vững” - ông Anthony Tan khẳng định trong thông điệp gửi đến nhân viên Grab.
'''

In [23]:
import re

In [24]:
def summarize(text):
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_.,-0123456789%]',' ',text)
    text_encoding = tokenizer(
        text,
        max_length = 512,
        padding = "max_length",
        truncation = True,
        return_attention_mask = True,
        add_special_tokens = True,
        return_tensors = "pt"
    )

    generated_ids = trained_model.model.generate(
        input_ids = text_encoding["input_ids"],
        attention_mask = text_encoding["attention_mask"],
        max_length = 128,
        num_beams = 2,
        repetition_penalty=2.5,
        length_penalty = 1.0,
        early_stopping = True
    )

    preds = [
        tokenizer.decode(gen_id, skip_special_tokens = True,clean_up_tokenization_spaces= True )
        for gen_id in generated_ids
    ]

    return "".join(preds)

In [25]:
model_summary = summarize(text)

In [26]:
model_summary

'Tập đoàn Công nghệ Grab vẫn duy trì chính sách hạn chế sa thải và chỉ giảm tốc tuyển dụng cho các vị trí quản lý cấp cao.'

In [18]:
test_df=pd.read_csv("DataTest_v4.csv")

In [19]:
test_df

,Unnamed: 0,Content,Summary
0,0,"Sáng 23.8 , Chủ tịch Nước Trần Đại Quang và P...",Chuyến thăm là dịp để lãnh đạo Việt Nam và các...
1,1,"Cuối tháng 10.2020 , từ phản ánh của người dân...","Sau trận mưa lũ lịch sử trong tháng 10.2020 , ..."
2,2,Họ rất thích cảnh đẹp Vịnh Rồng nhưng chê thậm...,"Tôi vừa tiễn một đoàn khách Pháp 10 người , đi..."
3,3,"Theo đó , bà Hồ Thu Ánh - Giám đốc Sở Lao động...","Ngày 3.12 , Hội đồng nhân dân HĐND tỉnh Hậu Gi..."
4,4,Trên đây là câu hỏi của bạn đọc có email maikh...,Anh chị tôi đã qua đời để lại một con nhỏ mới ...
...,...,...,...
3330,3330,"Ngày 12.6 , Chỉ huy Đồn Biên phòng Ba Sơn tỉnh...","Mua được số ma tuý , Vi Văn Hùng 30 tuổi , huy..."
3331,3331,"Chiều 5.4.2022 , tại Trụ sở Chính phủ , Thủ tư...","Thủ tướng Phạm Minh Chính đề nghị Panama , một..."
3332,3332,"Theo Sina ngày 19/4 , trong hôn lễ ở đảo Bali ...","Hà Siêu Liên , con gái Vua sòng bạc Macau , nó..."
3333,3333,"Theo đó , Anh đã đăng trên Facebook có tài kho...","Chiều ngày 1.4 , Phòng An ninh Chính trị nội b..."


In [20]:
pre=[]
testsum=[]

In [21]:
for i in range(0,3000):
    model_summa=summarize(test_df.Content[i])
    testsum.append(test_df.Summary[i])
    pre.append(model_summa)

In [22]:
from rouge import Rouge 
rouge = Rouge()
scores = rouge.get_scores(pre, testsum, avg=True)
scores

{'rouge-1': {'r': 0.28368404332481134,
  'p': 0.34885966313530314,
  'f': 0.30359600853565083},
 'rouge-2': {'r': 0.15567990700303233,
  'p': 0.19789248192383732,
  'f': 0.16785919162978566},
 'rouge-l': {'r': 0.24443836993401327,
  'p': 0.30110206811078666,
  'f': 0.26182998372888167}}